In [5]:
from connector import set_connection
import pandas as pd
import numpy as np
from collections import Counter

In [6]:
def run_query(query:str):
    with set_connection() as dd:
        return dd.query(query).to_df()    

In [7]:
def most_common_elements(arrays):
    element_counts = Counter(arrays)
    max_frequency = max(element_counts.values())
    return [element for element, count in element_counts.items() if count == max_frequency]

In [8]:
def get_cross_books(books_1, books_2):
    return list(set(books_1) & set(books_2))

In [9]:
def get_user_book_ratings(user_id:int):
    return run_query(f"""
    select
	    isbn
	    , book_rating
    from ratings
    where user_id = {user_id}
    """)

In [11]:
def get_top_rated_users(isbn:str, n_users:int):
    query = f"""
    select
        r.user_id
        , r.book_rating
        , count
        , mean_rating
    from ratings r
    left join ( 	
        select
        user_id
            , count(isbn) as count
            , avg(book_rating) as mean_rating 
        from ratings 
        group by user_id
    ) t on t.user_id=r.user_id
    where isbn = '{isbn}' and mean_rating > 4
    order by book_rating desc, count desc, mean_rating desc
    limit {n_users}
    """
    return run_query(query)

In [12]:
query_top_users = """
select *
from (
select
	r.user_id
	, avg(r.book_rating) as rating
	, count(b.isbn) as count
from ratings r
left join books b on b.isbn=r.isbn
group by r.user_id
order by count desc, rating desc
)
where rating > 4 and rating < 7
limit 200
"""

In [13]:
top10_users = run_query(query_top_users)['user_id']

In [14]:
users10_books = np.empty((0, ))

In [15]:
for user in top10_users:
    books = run_query(f"""
    select
	isbn
    from ratings
    where user_id = {user} and book_rating = 10
    """)['isbn']
    users10_books = np.append(users10_books, books)

In [126]:
model = {}
model['head'] = most_common_elements(users10_books)

In [129]:
model

{'head': ['043935806X']}

In [158]:
def train(model:dict, n_groups:int, n_users=30, n_books=20):
    for group in range(n_groups):
        isbn = model['head'][group]
        model[isbn] = {}      
        model[isbn]['users'] = get_top_rated_users(isbn, n_users)['user_id'].tolist()

        df = run_query('select * from ratings')
        
        
        model[isbn]['books'] = list(set(df[(df['user_id'].isin(model[isbn]['users'])) & 
                                         (df['book_rating'] > 8)]['isbn'].value_counts().index.tolist()))[:n_books]
        
        
        next_groups = df[(df['user_id'].isin(model[isbn]['users'])) & 
                         (df['book_rating'] <= 3)]['isbn'].value_counts().index.tolist()
        
        for ng in next_groups:
            if ng not in model['head']:
                model['head'].append(ng)
                break
        
    return model

In [159]:
model = train(model=model, n_groups=30, n_users=50, n_books=20)

In [162]:
len(model.keys())

76

In [139]:
model

{'head': ['043935806X',
  '0971880107',
  '059035342X',
  '0671027360',
  '0671027387',
  '0312995423',
  '0743403452',
  '0142001740',
  '0060928336',
  '0316666343',
  '0425167313',
  '0316601950',
  '0684872153',
  '0345350499',
  '0060977493',
  '0316776963',
  '0380789035',
  '0515130966',
  '0399147381',
  '0156028778',
  '0312422156',
  '0060930535',
  '0375706771',
  '0786868716',
  '0425144429',
  '0385504209',
  '0553802496',
  '0399151273',
  '0671672797',
  '0061094226',
  '0425160394',
  '0515124214',
  '0515121843',
  '051513628X',
  '0515136379',
  '0515136530',
  '0515132187',
  '0373484224',
  '0373484232',
  '0446516538',
  '0446606812',
  '0446608955',
  '0671695142',
  '0446605484',
  '0446612790',
  '0446610038',
  '0440236703',
  '0425189864',
  '0345452569',
  '0679746048',
  '0140244824',
  '0804106304',
  '0312195516',
  '0452282152',
  '0440211263',
  '0345351525',
  '0345384466',
  '0345397819',
  '0345409671',
  '034538475X',
  '0345417623',
  '0345353145',


In [105]:
import json

In [154]:
with open("model.json", "w") as json_file:
    json.dump(model, json_file)